# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 22 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 23). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la energía generada 24 horas después. 

Por lo tanto, vamos a comenzar con todos los imports necesarios para este proyecto

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor  
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn import neighbors

También, vamos a importar el documento con los datos de entrenamiento, y le vamos a eliminar las columnas innecesarias

In [ ]:
import pandas as pd


# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')

# FIlter the data to only include the columns that end in 13
data = data.filter(regex='13$|energy')
#print(data)




Y separar los datos de entrenamiento de los datos de test, con train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

y = data['energy']
x = data.drop(columns=['energy'])

# test_size vamos a escoger 0.2 ya que tenemos datos en un rango de 5 años en orden (2015-2019) por lo que usaremos 2019 como test final.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7543)


In [ ]:
# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier  
from sklearn.pipeline import Pipeline


scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer()]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), x.columns)
    ])


# Decidir usando KNN
A continuación, usaremos KNN 

# Entrenamiento de diferentes modelos
Lo próximo que haremos, será probar con diferentes métodos de entrenamiento (KNN, árboles de regresión, regresión lineal normal y la variante de Lasso, y SVM. Para elegir el mejor, usaremos la librería time para saber cuánto se tarda en entrenar cada modelo, un dummy para comparar resultados, y el mejor de entre ellos, usando una política de importancia de 20% a tiempo y 80% precisión.

Para empezar, vamos a probar con un Decision Tree Regresor.

In [ ]:
# Optimizing hyperparameters
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor  # Import DecisionTreeRegressor instead of DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
import numpy as np
import time


param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Adjust criterion for regression
    'max_depth': [None, 12, 13, 14, 15, 16, 17],
    'min_samples_split': range(2, 15, 2),
    'min_samples_leaf': range(5, 15, 2),
}

cv = KFold(n_splits=5)
model = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error', n_iter=50)  # Adjust scoring metric

model2 = GridSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


rmse_tree = metrics.mean_squared_error(y_test, y_pred)
sqrt_rmse_tree = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse_tree}")
print(f"SQRT RMSE: {sqrt_rmse_tree}")
print(f"Best params: {model.best_params_}")
# Best params: {'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 3}


In [ ]:
# Create a Dunny regressor to obtain a baseline

from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', dummy)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

rmse_dummy = metrics.mean_squared_error(y_test, y_pred)
sqrt_rmse_dummy = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse_dummy}")
print(f"SQRT RMSE: {sqrt_rmse_dummy}")



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV


param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1],  # -1 indica utilizar todos los núcleos
    'positive': [True, False]
}


# Crear el modelo de regresión lineal
model = LinearRegression()

grid_search = GridSearchCV(model,
                           param_grid,
                           cv=5,
                           scoring="neg_mean_absolute_percentage_error"
                           )

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)



Ahora que ya hemos probado con el Lineal Model normal, vamos a probar con la variante de Lasso a ver si conseguimos mejorar el entrenamiento:

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Definir los parámetros que deseas probar
parameters = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Valores de regularización
    'fit_intercept': [True, False],
    'copy_X': [True, False], # pendientes de borrar
    'positive': [True, False], 
    'precompute': [True, False], 
    'random_state': [7543], 
    'selection': ['cyclic', 'random'], 
    'tol': [1**(-10), 1**(-5), 1**(-4), 1**(-3), 1**(-2)], 
    'warm_start': [True, False]
}

# Crear el modelo de regresión lineal Lasso
model = Lasso()

grid_search = GridSearchCV(model, 
                           parameters, 
                           scoring='neg_mean_squared_error', 
                           cv=5)

grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)

In [ ]:
# Vamos a probar con KNN Regresor
from sklearn import neighbors
from sklearn import metrics
regr = neighbors.KNeighborsRegressor()

# We train it
regr.fit(X_train, y_train)

# We obtain predictions on the test set
y_test_pred = regr.predict(X_test)

# We compute accuracy
rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print(f"RMSE of the model: {rmse_knn}")

## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.